# Goal

* Conduct concenTRAIT on rhizosphere responds


# Init

In [1]:
import os
%load_ext rpy2.ipython

In [2]:
%%R
library(dplyr)
library(tidyr)
library(ggplot2)
library(ape)

/home/bryan/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/home/bryan/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/home/bryan/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)


# Setting variables

In [3]:
workDir = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/consenTRAIT'

# 16S tree file of all representative OTU sequences
treeFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/fasttree/aln_for_tree_boot.nwk'
rootName = 'X90478'

# log2 fold change
l2fcResDir = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/DeSeq2/'
l2fcResFile = os.path.join(l2fcResDir, 'Responders.txt')

# log2 fold change pvclust clusters
#pvclustDir = '/var/seq_data/fullCyc/MiSeq_16SrRNA/515f-806r/lib1-7rs/incorporators/l2fc_heavy_window/l2fc_pvclust_htmp/'
#pvclustFile = 'l2fc_sparse-maxRejH_pvclust.txt'

# params
padj_cutoff = 0.05  
nprocs = 24

In [4]:
if not os.path.isdir(workDir):
    os.makedirs(workDir)
%cd $workDir

/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/consenTRAIT


# Testing trait depth on rhizosphere responders

### prune tree to OTUs in responder file

In [ ]:
%%R -i treeFile -i workDir -i rootName

trees = read.tree(treeFile)
trees

In [ ]:
%%R -i l2fcResFile
#Read responder table in order to prune trees
Resp = read.delim(l2fcResFile, sep='\t', header = FALSE)
head(Resp)

Resp$V1 = Resp$V1 %>% as.vector
rootrow = c(rootName, rep(0, ncol(Resp)-1))
Resp = rbind(Resp, rootrow)

write.table(Resp, 'Responders_wRoot.txt', sep='\t', quote=FALSE, row.names=FALSE, col.names=FALSE)
cat('Total number of OTUs:', Resp$V1 %>% unique %>% length, '\n\n')
Resp%>% tail(n=3)

In [ ]:
%%R
Taxa = as.character(Resp)


In [36]:
%%R

pruneTree = function(tree, taxa){
    # not matching OTUs
    to.rm = setdiff(tree$tip, taxa)
    # pruning
    tree = drop.tip(tree, to.rm)    
    # rooting
    tree = multi2di(tree)
    tree = root(tree, rootName)
    # returning
    return(tree)
    }

trees.p = lapply(trees, pruneTree, taxa=Taxa)
class(trees.p) = 'multiPhylo'
trees.p

100 phylogenetic trees


In [41]:
%%R
# writing pruned tree
tree.out = file.path(workDir, 'otusn_pick_3in3.nwk')
write.tree(trees.p, tree.out)
cat('File written:', tree.out, '\n')

File written: /home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/consenTRAIT/otusn_pick_3in3.nwk 


In [42]:
%%R
# clearing memory
trees = trees.p = NULL

# Calling concenTRAIT - Nick's old version

In [46]:
treeFile = "/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/consenTRAIT/otusn_pick_3in3.nwk"
respFileDir = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/consenTRAIT/'
respFile = os.path.join(respFileDir, 'Responders_wRoot.txt')

In [ ]:
!/home/bryan/tree_edit/bin/concentrait.r -p 20 \
    $treeFile $rootName $respFile > RhizResponders_tauD.txt

Analyzing tree:  1 
  Analyzing trait:  1 
Analyzing tree:  2 
  Analyzing trait:  1 
Analyzing tree:  3 
  Analyzing trait:  1 
Analyzing tree:  4 
  Analyzing trait:  1 
Analyzing tree:  5 
  Analyzing trait:  1 
Analyzing tree:  6 
  Analyzing trait:  1 
Analyzing tree:  7 
  Analyzing trait:  1 
Analyzing tree:  8 
  Analyzing trait:  1 
Analyzing tree:  9 
  Analyzing trait:  1 
Analyzing tree:  10 
  Analyzing trait:  1 
Analyzing tree:  11 
  Analyzing trait:  1 
Analyzing tree:  12 
  Analyzing trait:  1 
Analyzing tree:  13 
  Analyzing trait:  1 
Analyzing tree:  14 
  Analyzing trait:  1 
Analyzing tree:  15 
  Analyzing trait:  1 
Analyzing tree:  16 
  Analyzing trait:  1 
Analyzing tree:  17 
  Analyzing trait:  1 
Analyzing tree:  18 
  Analyzing trait:  1 
Analyzing tree:  19 
  Analyzing trait:  1 
Analyzing tree:  20 
  Analyzing trait:  1 
Analyzing tree:  21 
  Analyzing trait:  1 
Analyzing tree:  22 
  Analyzing trait:  1 
Analyzing tree:  23 
  Analyzing trait:  

Trait	  tau_D			p-value 
t1	0.020504968504344	0.04 